In [30]:
from anthropic import Anthropic
import os
from dotenv import load_dotenv

load_dotenv()

True

#### Use Anthropic key

In [31]:
api_key = os.getenv('ANTHROPIC_API_KEY')
base_url = os.getenv('ANTHROPIC_BASE_URL')

In [33]:
client = Anthropic(api_key=api_key,base_url=base_url)
response = client.messages.create(max_tokens=1000,model='claude-3-5-sonnet-20240620',messages=[{'role':'user','content':'你是谁'}])
print(response.content[0].text)

我是一个人工智能助手,由Anthropic公司开发。我的名字是Claude。很高兴认识你!我可以回答问题、提供信息和帮助完成各种任务。有什么我可以为你做的吗?


In [34]:
def llm_call(prompt:str,system_prompts:str="",model='claude-3-5-sonnet-20240620')->str:
    client = Anthropic(api_key=api_key,base_url=base_url)
    messages = [{'role':'user','content':prompt}]
    response = client.messages.create(max_tokens=1000,model=model,messages=messages,system=system_prompts,temperature=0.1)
    return response.content[0].text

In [16]:
llm_call('你是谁')

'我是 Claude,一个 AI 助手。我会用专业、友善和诚实的态度来帮助你。'

In [36]:
import re
def extract_xml(text: str, tag: str) -> str:
    """
    Extracts the content of the specified XML tag from the given text. Used for parsing structured responses 

    Args:
        text (str): The text containing the XML.
        tag (str): The XML tag to extract content from.

    Returns:
        str: The content of the specified XML tag, or an empty string if the tag is not found.
    """
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""

In [35]:
from typing import Dict

def route(input:str,routes:Dict[str,str])->str:
    '''Route input to specialized prompt using content classification'''
    print(f'\nAvailable routes: {list(routes.keys())}')
    selector_prompt = f'''
    Analyze the input and select the most appropriate support team from these options:{list(routes.keys())}
    First explain your reasoning, then provide your selection in this this XML format:
    
    <reasoning>
    Brief explanation of why this ticket should be routed to a specific team.
    Consider key terms, user intent, and urgency level. 
    </reasoning>
    
    <selection>
    The chosen team name
    </selection>
    
    Input:{input}
    '''.strip()
    
    route_response = llm_call(selector_prompt)
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response,'selection').strip().lower()
    
    print(f'Routing Analysis:')
    print(reasoning)
    print(f'Selected route:{route_key}')
    
    selected_prompt = routes[route_key]
    # return llm_call(f"{selected_prompt}\nInput: {input}")
    return llm_call(prompt=input,system_prompts=selected_prompt)
    

In [37]:
# Route support tickets to appropriate teams based on content analysis

support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant
    
    Keep responses professional but friendly.
    
    Input: """,
    
    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed
    
    Use clear, numbered steps and technical details.
    
    Input: """,
    
    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone.
    
    Input: """,
    
    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help
    
    Be educational and encouraging in tone.
    
    Input: """
}

In [38]:
# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John""",
    
    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",
    
    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    print(f"\nTicket {i}:")
    print("-" * 40)
    print(ticket)
    print("\nResponse:")
    print("-" * 40)
    response = route(ticket, support_routes)
    print(response)

Processing support tickets...


Ticket 1:
----------------------------------------
Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John

Response:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
Routing Analysis:

This issue is clearly related to account access problems. The user is unable to log in despite being confident about using the correct password. This type of problem typically falls under account management and security, which is handled by the account team. The issue doesn't appear to be related to billing, product features, or technical problems with the system itself. The urgency expressed by the user ("This is urgent") further supports routing this to the account team, as they are

### summary

最后一个问题Ticket 3关于导出Excel的问题，使用claude-0620模型将该路由转到了product route，官网使用的claude-1022模型转到了technical route。

分析差异：

在这个示例中，并没有给出每个路由具体的职责，只给出了指示该路由如何回复用户问题

因此模型只能根据用户的问题，将内容对应到可用路由的名称上来判断其归属，因此可能存在模糊边界。